In [ ]:
# Sender Block 1: Install and Import Libraries
!pip install cryptography
from cryptography.hazmat.primitives import hashes, padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import padding as asym_padding
from cryptography.hazmat.primitives import serialization
import os

print("Libraries installed and imported successfully!")

Libraries installed and imported successfully!


In [ ]:
# Sender Block 2: Load RSA Public Key
def load_public_key(public_key_path):
    try:
        with open(public_key_path, "rb") as f:
            return serialization.load_pem_public_key(f.read())
    except Exception as e:
        print(f"Error loading public key: {e}")
        return None

print("Public key loading function defined. Provide the path in the next step.")

Public key loading function defined. Provide the path in the next step.


In [ ]:
# Sender Block 3: Encryption Function
def encrypt_file(input_file, output_file, public_key):
    try:
        aes_key = os.urandom(32)
        camellia_key = os.urandom(32)
        aes_iv = os.urandom(16)
        camellia_iv = os.urandom(16)

        with open(input_file, "rb") as f:
            plaintext = f.read()
        split_point = len(plaintext) // 2
        part1, part2 = plaintext[:split_point], plaintext[split_point:]

        padder1 = padding.PKCS7(128).padder()
        padded_part1 = padder1.update(part1) + padder1.finalize()
        padder2 = padding.PKCS7(128).padder()
        padded_part2 = padder2.update(part2) + padder2.finalize()

        aes_cipher = Cipher(algorithms.AES(aes_key), modes.CBC(aes_iv))
        aes_encryptor = aes_cipher.encryptor()
        aes_ciphertext = aes_encryptor.update(padded_part1) + aes_encryptor.finalize()

        camellia_cipher = Cipher(algorithms.Camellia(camellia_key), modes.CBC(camellia_iv))
        camellia_encryptor = camellia_cipher.encryptor()
        camellia_ciphertext = camellia_encryptor.update(padded_part2) + camellia_encryptor.finalize()

        encrypted_aes_key = public_key.encrypt(
            aes_key,
            asym_padding.OAEP(mgf=asym_padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        encrypted_camellia_key = public_key.encrypt(
            camellia_key,
            asym_padding.OAEP(mgf=asym_padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )

        aes_length = len(aes_ciphertext).to_bytes(4, byteorder='big')
        camellia_length = len(camellia_ciphertext).to_bytes(4, byteorder='big')

        with open(output_file, "wb") as f:
            f.write(encrypted_aes_key)
            f.write(encrypted_camellia_key)
            f.write(aes_iv)
            f.write(camellia_iv)
            f.write(aes_length)
            f.write(camellia_length)
            f.write(aes_ciphertext)
            f.write(camellia_ciphertext)

        print(f"File encrypted and saved as {output_file}")
    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
    except Exception as e:
        print(f"Error during encryption: {e}")

In [ ]:
# Sender Block 4: Sender’s Main Execution
# Upload your file (e.g., text.txt or image.jpg) and public_key.pem to /content/
input_file = input("Enter file to encrypt (e.g., /content/text.txt or /content/image.jpg): ").strip()
output_file = input("Enter encrypted output file path (e.g., /content/encrypted_file.bin): ").strip()
public_key_path = input("Enter RSA public key file path (e.g., /content/public_key.pem): ").strip()

public_key = load_public_key(public_key_path)
if public_key:
    encrypt_file(input_file, output_file, public_key)
    print("Encryption complete. Share the encrypted file with the receiver.")
else:
    print("Encryption aborted due to public key loading failure.")

Enter file to encrypt (e.g., /content/text.txt or /content/image.jpg): /content/bigdataass2.jpg
Enter encrypted output file path (e.g., /content/encrypted_file.bin): /content/encrypted_file.bin
Enter RSA public key file path (e.g., /content/public_key.pem): /content/public_key.pem
File encrypted and saved as /content/encrypted_file.bin
Encryption complete. Share the encrypted file with the receiver.
